In [ ]:
import json
import ijson

with open('allowed_words.txt') as f:
    allowed_words = f.read().splitlines()
                                
with open('freq_map.json') as f:
    freq_map = json.load(f)
    
possible_pattern = {}
with open('possible_pattern.json') as f:
    parser = ijson.parse(f)
    for prefix, event, value in parser:
        if event == "start_map" and prefix != "":
            possible_pattern[prefix] = {}
        elif event == "string":
            possible_pattern[prefix[0:5]][prefix[6:11]] = value

In [ ]:
def hardMode(possible_words, guess_num, max_p):
    total_p = 0
    for i in possible_words:
        total_p += freq_map[i]
    
    best_guess = " "*5
    best_expect = 7
    if total_p == 0:
        best_expect = 7
    else:
        best_expect = max_p/total_p
    
    for guess in possible_words:
        if guess_num < 6:
            ## build the freq dict
            freq_dict = {}
            for answer in possible_words:
                result = possible_pattern[guess][answer]
                if result in freq_dict:
                    freq_dict[result].append(answer)
                else:
                    freq_dict[result] = [answer]
            
            cum_p = 0
            expect = 0
            for k, v in freq_dict.items():
                if total_p == 0:
                    min_expect = expect
                else:
                    min_expect = expect + 1 - cum_p/total_p
                    
                if min_expect > best_expect:
                    expect = min_expect
                    break
                
                if k == "GGGGG":
                    P = freq_map[v[0]]
                    G = v[0];
                    E = 0;
                elif len(v) == 1:
                    P = freq_map[v[0]]
                    G = v[0];
                    E = 1;
                elif len(v) == 2:
                    P = freq_map[v[0]] + freq_map[v[1]]
                    if P == 0:
                        G = v[0]
                        E = 3/2
                    elif freq_map[v[0]] > freq_map[v[1]]:
                        G = v[0];
                        E = 1+freq_map[v[1]]/P;
                    else:
                        G = v[1];
                        E = 1+freq_map[v[0]]/P;
                else:
                    P, G, E = hardMode(v, guess_num+1, (best_expect-expect)*total_p)
                    
                cum_p += P
                if total_p == 0:
                    expect += len(v)/len(possible_words) * (E+1)
                else:
                    expect += P/total_p * (E+1)
        else:
            if total_p == 0:
                expect = 1 + 1-1/len(possible_words)
            else:
                expect = 1 + 1-freq_map[guess]/total_p
            
        if expect < best_expect:
            best_guess = guess
            best_expect = expect

    return total_p, best_guess, best_expect

In [ ]:
possible_words = allowed_words

print("Example: ROATE 🟨⬛🟩⬛⬛ input: roate YBGBB")

guess = ' '*11

num = 0
while not (guess[6:11] == "GGGGG"):
    num += 1
    next_possible_words = []

    guess = input()
    for char in guess:
        if char == 'G':
            print("🟩", end='')
        elif char == 'Y':
            print("🟨", end='')
        elif char == 'B':
            print("⬛", end='')
        else:
            print(char, end='')
    print()

    for word in possible_words:
        if possible_pattern[guess[0:5]][word] == guess[6:11]:
            next_possible_words.append(word)
    possible_words = next_possible_words
    
    print("# of possible solutions :", len(possible_words))
    
    P, G, E = hardMode(possible_words, num+1, 7)
    print(G, E)

In [ ]:
## finding start word

TOTAL_P = 0.079688853104321

P, G, E = hardMode(allowed_words, 1, 7*TOTAL_P)
print(G, E)

## entropy method
* total entropy: 9.243309925972358
* top 5 start word:
    * ('tares', 5.96463766256561)
    * ('tears', 5.907751486245412)
    * ('tries', 5.90176559468474)
    * ('tores', 5.900578491569622)
    * ('stare', 5.855530842375718)
    
## brute force method
* ('tears', 2.8521390723896127)
* ('tares', 2.85717712365544)
* ('taser', 2.874560888006384)
* ('salet', 2.8937745301017723)

tries
tores
stare

roate
raise

crane